<a href="https://colab.research.google.com/github/dwarcy/TopicosEspeciais_IA/blob/main/Aula_3_Transformer_Decoder_only_and_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 3 - Transformer *decoder-only*

Nesta aula você irá modificar o Transformer *decoder-only* fornecido a seguir.
Observe que em um *decoder-only* não existe:

*   cross-attention
*   encoder separado

e é utilizado com *auto-regressão*.

## Objetivo


## Exercício

Neste exercício você deve:

1.   carregar o seu conjunto de documentos
2.   treinar e usar (ou carregar) um tokenizador
3.   fazer treino de um modelo decoder-only
4.   incluir no loop de treino, inferência usando máxima probabilidade
5.   incluir no loop de treino, inferência usando amostragem com temperatura


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [6]:
import json

entrada = "/content/documentos1.jsonl"
saida = "/content/documentos_corrigido.jsonl"

with open(entrada, "r", encoding="utf-8") as f_in, open(saida, "w", encoding="utf-8") as f_out:
    # Pular a primeira linha (o cabeçalho 'description')
    linhas = f_in.readlines()[1:]

    for linha in linhas:
        texto = linha.strip()
        if texto:  # Ignora linhas vazias
            # Cria o dicionário e escreve como uma string JSON em uma nova linha
            json_line = json.dumps({"description": texto}, ensure_ascii=False)
            f_out.write(json_line + "\n")

print("Arquivo corrigido com sucesso!")

Arquivo corrigido com sucesso!


# Passo 1: carregar conjunto de documentos

In [43]:
from datasets import load_dataset
import re

###### INSIRA AQUI O CODIGO PARA CARREGAR OS SEUS DOCUMENTOS na lista DOCUMENTOS
#ds = load_dataset("jquigl/imdb-genres") # Exemplozz
# ds = load_dataset("/content/documentos1.jsonl")
ds = load_dataset("json", data_files="/content/documentos_corrigido.jsonl")

def clean_ascii(text):
    # text = text.encode("ascii", errors="ignore").decode()
    text = text.lower()
    return re.sub(r"[^a-z0-9 .,:;!?'\-áàâãéèêíïóôõúç]", "", text)

# Extract documentos
documentos = [clean_ascii(x["description"]) for x in ds["train"]]
documentos = [t.split(" - ")[0] for t in documentos]   # optional remove year
documentos = [t for t in documentos if len(t) > 0]

#documentos = [ "meu doc favorito 1", "meu doc menos favorito 2"]
print("Total documentos:", len(documentos))
print("Sample:", documentos[:10])

Total documentos: 62
Sample: ['eu não sei o que isso quer dizer.', 'naquela mesa ele sentava sempre.', 'um velho cruza a soleira.', 'existirmos: a que será que se destina?', 'uma luz azul me guia.', 'baby, dê-me seu dinheiro que eu quero viver.', 'ando por aí querendo te encontrar.', 'vou te contar que os olhos já nem podem ver.', 'eu sei que determinada rua que eu já passei não tornará a ouvir o som dos meus passos.', 'da manga rosa, quero o gosto e o sumo.']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Passo 2: Carregar ou treinar um tokenizador

In [44]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
import random
import re

##### Insira aqui o código para treinar o seu TOKENIZER
# Defina o seu tokenizador
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordLevelTrainer(
    vocab_size=1000,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"]
)
# Treino do tokenizador
tokenizer.train_from_iterator(documentos, trainer)

# Funções auxiliares para transitar entre tokens textuais e ids de tokens
def encode(text):
    ids = tokenizer.encode("[BOS] " + text + " [EOS]").ids
    return torch.tensor(ids, dtype=torch.long)

def decode(ids):
    return tokenizer.decode(ids.tolist())

vocab_size = tokenizer.get_vocab_size()

# Pega o dicionário {palavra: id}
vocab = tokenizer.get_vocab()

# Ordena pelo ID (do 0 em diante)
vocab_ordenado = sorted(vocab.items(), key=lambda item: item[1])

print("--- Mapeamento Completo (Primeiros 20) ---")
for palavra, token_id in vocab_ordenado[:100]:
    print(f"ID: {token_id} | Palavra: {palavra}")

--- Mapeamento Completo (Primeiros 20) ---
ID: 0 | Palavra: [PAD]
ID: 1 | Palavra: [UNK]
ID: 2 | Palavra: [BOS]
ID: 3 | Palavra: [EOS]
ID: 4 | Palavra: .
ID: 5 | Palavra: ,
ID: 6 | Palavra: que
ID: 7 | Palavra: a
ID: 8 | Palavra: de
ID: 9 | Palavra: o
ID: 10 | Palavra: e
ID: 11 | Palavra: não
ID: 12 | Palavra: se
ID: 13 | Palavra: uma
ID: 14 | Palavra: é
ID: 15 | Palavra: do
ID: 16 | Palavra: um
ID: 17 | Palavra: -
ID: 18 | Palavra: em
ID: 19 | Palavra: mais
ID: 20 | Palavra: mas
ID: 21 | Palavra: me
ID: 22 | Palavra: eu
ID: 23 | Palavra: você
ID: 24 | Palavra: as
ID: 25 | Palavra: os
ID: 26 | Palavra: ?
ID: 27 | Palavra: da
ID: 28 | Palavra: mundo
ID: 29 | Palavra: como
ID: 30 | Palavra: para
ID: 31 | Palavra: com
ID: 32 | Palavra: na
ID: 33 | Palavra: no
ID: 34 | Palavra: quando
ID: 35 | Palavra: das
ID: 36 | Palavra: ele
ID: 37 | Palavra: homem
ID: 38 | Palavra: mim
ID: 39 | Palavra: olhos
ID: 40 | Palavra: por
ID: 41 | Palavra: sem
ID: 42 | Palavra: sempre
ID: 43 | Palavra: sua
ID:

# TREINANDO COM BPE
agrupa caracteres comuns em sub-palavras.

In [45]:
# @title
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
import random
import re

# 1. Altere o Modelo
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# 2. Altere o Trainer
trainer = BpeTrainer(
    vocab_size=2000,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"]
)

# Treino do tokenizador
tokenizer.train_from_iterator(documentos, trainer)

# Funções auxiliares para transitar entre tokens textuais e ids de tokens
def encode(text):
    ids = tokenizer.encode("[BOS] " + text + " [EOS]").ids
    return torch.tensor(ids, dtype=torch.long)

def decode(ids):
    return tokenizer.decode(ids.tolist())

vocab_size = tokenizer.get_vocab_size()

# Pega o dicionário {palavra: id}
vocab = tokenizer.get_vocab()

# Ordena pelo ID (do 0 em diante)
vocab_ordenado = sorted(vocab.items(), key=lambda item: item[1])

print("--- Mapeamento Completo (Primeiros 20) ---")
for palavra, token_id in vocab_ordenado:
    print(f"ID: {token_id} | Palavra: {palavra}")

--- Mapeamento Completo (Primeiros 20) ---
ID: 0 | Palavra: [PAD]
ID: 1 | Palavra: [UNK]
ID: 2 | Palavra: [BOS]
ID: 3 | Palavra: [EOS]
ID: 4 | Palavra: ,
ID: 5 | Palavra: -
ID: 6 | Palavra: .
ID: 7 | Palavra: :
ID: 8 | Palavra: ;
ID: 9 | Palavra: ?
ID: 10 | Palavra: a
ID: 11 | Palavra: b
ID: 12 | Palavra: c
ID: 13 | Palavra: d
ID: 14 | Palavra: e
ID: 15 | Palavra: f
ID: 16 | Palavra: g
ID: 17 | Palavra: h
ID: 18 | Palavra: i
ID: 19 | Palavra: j
ID: 20 | Palavra: l
ID: 21 | Palavra: m
ID: 22 | Palavra: n
ID: 23 | Palavra: o
ID: 24 | Palavra: p
ID: 25 | Palavra: q
ID: 26 | Palavra: r
ID: 27 | Palavra: s
ID: 28 | Palavra: t
ID: 29 | Palavra: u
ID: 30 | Palavra: v
ID: 31 | Palavra: x
ID: 32 | Palavra: y
ID: 33 | Palavra: z
ID: 34 | Palavra: à
ID: 35 | Palavra: á
ID: 36 | Palavra: â
ID: 37 | Palavra: ã
ID: 38 | Palavra: ç
ID: 39 | Palavra: é
ID: 40 | Palavra: ê
ID: 41 | Palavra: í
ID: 42 | Palavra: ó
ID: 43 | Palavra: ô
ID: 44 | Palavra: õ
ID: 45 | Palavra: ú
ID: 46 | Palavra: as
ID: 47 | P

# Treinar com WordPiece
é similar ao BPE, mas usa um critério de probabilidade para decidir o que agrupar.

In [57]:
# @title
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
import random
import re

# 1. Altere o Modelo
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# 2. Altere o Trainer
trainer = WordPieceTrainer(
    vocab_size=2000,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"]
)

# Treino do tokenizador
tokenizer.train_from_iterator(documentos, trainer)

# Funções auxiliares para transitar entre tokens textuais e ids de tokens
def encode(text):
    ids = tokenizer.encode("[BOS] " + text + " [EOS]").ids
    return torch.tensor(ids, dtype=torch.long)

def decode(ids):
    return tokenizer.decode(ids.tolist())

vocab_size = tokenizer.get_vocab_size()

# Pega o dicionário {palavra: id}
vocab = tokenizer.get_vocab()

# Ordena pelo ID (do 0 em diante)
vocab_ordenado = sorted(vocab.items(), key=lambda item: item[1])

print("--- Mapeamento Completo (Primeiros 20) ---")
for palavra, token_id in vocab_ordenado[:100]:
    print(f"ID: {token_id} | Palavra: {palavra}")

--- Mapeamento Completo (Primeiros 20) ---
ID: 0 | Palavra: [PAD]
ID: 1 | Palavra: [UNK]
ID: 2 | Palavra: [BOS]
ID: 3 | Palavra: [EOS]
ID: 4 | Palavra: ,
ID: 5 | Palavra: -
ID: 6 | Palavra: .
ID: 7 | Palavra: :
ID: 8 | Palavra: ;
ID: 9 | Palavra: ?
ID: 10 | Palavra: a
ID: 11 | Palavra: b
ID: 12 | Palavra: c
ID: 13 | Palavra: d
ID: 14 | Palavra: e
ID: 15 | Palavra: f
ID: 16 | Palavra: g
ID: 17 | Palavra: h
ID: 18 | Palavra: i
ID: 19 | Palavra: j
ID: 20 | Palavra: l
ID: 21 | Palavra: m
ID: 22 | Palavra: n
ID: 23 | Palavra: o
ID: 24 | Palavra: p
ID: 25 | Palavra: q
ID: 26 | Palavra: r
ID: 27 | Palavra: s
ID: 28 | Palavra: t
ID: 29 | Palavra: u
ID: 30 | Palavra: v
ID: 31 | Palavra: x
ID: 32 | Palavra: y
ID: 33 | Palavra: z
ID: 34 | Palavra: à
ID: 35 | Palavra: á
ID: 36 | Palavra: â
ID: 37 | Palavra: ã
ID: 38 | Palavra: ç
ID: 39 | Palavra: é
ID: 40 | Palavra: ê
ID: 41 | Palavra: í
ID: 42 | Palavra: ó
ID: 43 | Palavra: ô
ID: 44 | Palavra: õ
ID: 45 | Palavra: ú
ID: 46 | Palavra: ##i
ID: 47 | 

## Definição do modelo Transformer Decoder-only

In [58]:
import torch
import torch.nn as nn

class DecoderOnlyTransformer(nn.Module):
    "Implementação de um transformer que tem somente a parte do decoder"
    def __init__(self, vocab_size, d_model=128, n_heads=4, num_layers=3, max_len=64):
        super().__init__()
        self.max_len = max_len

        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(max_len, d_model)

        layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=256,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(layer, num_layers=num_layers)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        B, T = x.shape
        pos = torch.arange(T, device=x.device).unsqueeze(0)

        h = self.token_emb(x) + self.pos_emb(pos)

        # Máscara causal
        mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()

        out = self.decoder(h, h, tgt_mask=mask)
        logits = self.lm_head(out)
        return logits

# Códigos de inferência

## Código de inferência simples: token com maior probabilidade


In [59]:
def max_prob_sampling(logits):
    next_token = logits.argmax(dim=-1)
    return next_token.unsqueeze(0)

## Código de inferência avançada: amostragem com temperatura

In [60]:
import torch.nn.functional as F
import torch

# Inferência (com temperature e top_p)
def sampling(logits, top_p=0.9, top_k=None, temperature=1.0):
    # Ajusta pela temperatura
    logits = logits / temperature

    # Se top_k for especificado, filtra por top_k primeiro
    if top_k is not None:
        # Ensure top_k is not larger than the vocabulary size
        k_to_use = min(top_k, logits.size(-1))
        # Get the top_k values and indices
        v, _ = torch.topk(logits, k_to_use)
        # Set logits of all values smaller than the k-th value to -inf
        logits[logits < v[:, [-1]]] = float('-inf')

    # Ordena os logits
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.softmax(sorted_logits, dim=-1).cumsum(dim=-1)

    # Mascara tokens acima do top_p
    mask = cumulative_probs > top_p
    # Garante que ao menos um token permaneça
    mask[..., 1:] = mask[..., :-1].clone()
    mask[..., 0] = False

    filtered_logits = sorted_logits.masked_fill(mask, float('-inf'))
    probs = torch.softmax(filtered_logits, dim=-1)

    # Amostra o token
    sampled_idx = torch.multinomial(probs, num_samples=1)

    # Converte para índice na tabela original
    next_token = sorted_indices[sampled_idx]

    return next_token


def generate(prompt, next_token_function, max_new_tokens=20, top_k=None, top_p=0.9, temperature=1.0):
    model.eval()
    x = encode(prompt).unsqueeze(0).to(device)

    for _ in range(max_new_tokens):
        logits = model(x)[:, -1, :]  # pega apenas o último passo

        # Passa os parâmetros de amostragem se a função for top_p_sampling
        if next_token_function == sampling:
            next_token = next_token_function(logits.squeeze(0), top_k=top_k, top_p=top_p, temperature=temperature)
        else:
            next_token = next_token_function(logits.squeeze(0))

        x = torch.cat([x, next_token.unsqueeze(0)], dim=1)

        if next_token.item() == tokenizer.token_to_id("[EOS]"):
            break

    return decode(x[0])


# 4. Fazer treino de modelo: códigos de treino

In [64]:
# Função auxiliar para gerar batches de exemplos para treino
def sample_batch(batch_size=16, max_len=20):
    batch = random.sample(documentos, batch_size)
    tokenized = [encode(t) for t in batch]

    max_t = min(max(len(x) for x in tokenized), max_len)
    padded = []

    for x in tokenized:
        x = x[:max_t]
        pad_len = max_t - len(x)
        if pad_len > 0:
            x = torch.cat([x, torch.zeros(pad_len, dtype=torch.long)])
        padded.append(x)

    return torch.stack(padded)

################################
device = "cuda" if torch.cuda.is_available() else "cpu"
model = DecoderOnlyTransformer(vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
#################################

steps = 10000

for step in range(1, steps + 1):
    model.train()
    batch = sample_batch().to(device)

    logits = model(batch[:, :-1])
    loss = F.cross_entropy(
        logits.reshape(-1, vocab_size),
        batch[:, 1:].reshape(-1),
        ignore_index=tokenizer.token_to_id("[PAD]")
    )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 50 == 0:
        ppl = torch.exp(loss).item()
        print(f"[step {step}] loss={loss.item():.4f}, ppl={ppl:.2f}")

    if step % 100 == 0:
        print("Generated text:")
        print(generate("eu sei ", next_token_function=sampling, top_p=0.9, temperature=0.8))
        print("--------------------------------------")
print("Training completed.")

[step 50] loss=7.1391, ppl=1260.26
[step 100] loss=7.0179, ppl=1116.46
Generated text:
eu sei ##mente ##ens explicar facil ##ter intelig ##ura angustiado todas ra ##éria can ##bula ju ##teiro objetivo busca ##ul ##ica agrade
--------------------------------------
[step 150] loss=6.9436, ppl=1036.47
[step 200] loss=6.7751, ppl=875.81
Generated text:
eu sei acha ##ul como u ##nca me ##imigo ##ti cima úteis aquelas mesqu ##ém ##nica intelig virá ##imáveis ma tingia inal
--------------------------------------
[step 250] loss=6.6362, ppl=762.18
[step 300] loss=6.5228, ppl=680.46
Generated text:
eu sei ##mor inteira amaria eram passado inteligente velho ##er guardiãs fer est que ide pernas
--------------------------------------
[step 350] loss=6.3722, ppl=585.32
[step 400] loss=6.3094, ppl=549.72
Generated text:
eu sei ##garia mim mo casaco claro esperança atrás um honestos . úte fa repende ##ne ##sa ##tasse te teme encont
--------------------------------------
[step 450] loss=6.2173, ppl=50

# Exercício: controle de temperatura, Top-K e Top-P

Modifique o código a seguir para fazer a visualização de produção de tokens do modelo que você treinou.

In [39]:
# @title
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

# -----------------------------
# Mock language-model logits
# -----------------------------
TOKENS = ["blue", "purple", "violet", "vio", "not", "Blue", "green", "gray", "grey", "black"]
BASE_LOGITS = np.array([6.0, 1.5, 0.5, 0.2, 0.2, 0.0, -5.0, -5.0, -5.0, -5.0])

def softmax(x):
    e = np.exp(x - np.max(x))
    return e / e.sum()

def apply_temperature(logits, temperature):
    return logits / max(temperature, 1e-5)

def apply_top_k(probs, k):
    if k >= len(probs):
        return probs
    idx = np.argsort(probs)[::-1]
    mask = np.zeros_like(probs)
    mask[idx[:k]] = 1
    probs = probs * mask
    return probs / probs.sum()

def apply_top_p(probs, p):
    idx = np.argsort(probs)[::-1]
    cumulative = np.cumsum(probs[idx])
    mask = cumulative <= p
    mask[np.argmax(mask)] = True
    new_probs = np.zeros_like(probs)
    new_probs[idx[mask]] = probs[idx[mask]]
    return new_probs / new_probs.sum()

# -----------------------------
# Widgets
# -----------------------------
prompt_dropdown = widgets.Dropdown(
    options=["Roses are red, violets are..."],
    value="Roses are red, violets are...",
    description="Prompt",
    layout=widgets.Layout(width="95%")
)

temperature_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description="Temperatura",
    readout_format=".1f",
    layout=widgets.Layout(width="90%")
)

topk_slider = widgets.IntSlider(
    value=6, min=1, max=10, step=1,
    description="Top-K",
    layout=widgets.Layout(width="90%")
)

topp_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=1.0, step=0.05,
    description="Top-P",
    readout_format=".2f",
    layout=widgets.Layout(width="90%")
)

# -----------------------------
# Plot
# -----------------------------
output_plot = widgets.Output()

def update_plot(*args):
    with output_plot:
        output_plot.clear_output()

        logits = apply_temperature(BASE_LOGITS, temperature_slider.value)
        probs = softmax(logits)
        probs = apply_top_k(probs, topk_slider.value)
        probs = apply_top_p(probs, topp_slider.value)

        fig, ax = plt.subplots(figsize=(10, 4))
        bars = ax.bar(TOKENS, probs * 100)
        ax.set_ylim(0, 100)
        ax.set_ylabel("Probabilidade (%)")
        ax.set_title("Probabilidade do próximo token")

        for bar, p in zip(bars, probs):
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                bar.get_height(),
                f"{p*100:.2f}%",
                ha="center",
                va="bottom",
                fontsize=9
            )

        plt.xticks(rotation=0)
        plt.show()

for w in [temperature_slider, topk_slider, topp_slider]:
    w.observe(update_plot, names="value")

# -----------------------------
# Collapsible explanation
# -----------------------------
accordion = widgets.Accordion(
    children=[widgets.HTML(
        """
        <ul>
          <li><b>Temperatura</b>: Controla aleatoriedade com mudança na escala dos logits.</li>
          <li><b>Top-K</b>: Restringe a amostra aos K tokens mais prováveis.</li>
          <li><b>Top-P</b>: Usa somente os menor conjunto de tokens que resultam em probabilidade acumulada até P.</li>
        </ul>
        """
    )]
)
accordion.set_title(0, "Entenda a visualização")

# -----------------------------
# Layout
# -----------------------------
controls = widgets.VBox([
    prompt_dropdown,
    widgets.HBox([temperature_slider]),
    widgets.HBox([topk_slider]),
    widgets.HBox([topp_slider])
])

display(Markdown("## Visualização de controle de Temperatura, Top-K e Top-P."))
display(widgets.VBox([
    widgets.HTML("<b>Parameters</b>"),
    controls,
    output_plot,
    accordion
]))

update_plot()


## Visualização de controle de Temperatura, Top-K e Top-P.

In [65]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
import torch

# ---------------------------------------------------------
# 1. FUNÇÕES DE SUPORTE (Matemática de Amostragem)
# ---------------------------------------------------------
def softmax(x):
    e = np.exp(x - np.max(x))
    return e / e.sum()

def apply_temperature(logits, temperature):
    return logits / max(temperature, 1e-5)

def apply_top_k(probs, k):
    if k >= len(probs): return probs
    idx = np.argsort(probs)[::-1]
    mask = np.zeros_like(probs)
    mask[idx[:k]] = 1
    probs = probs * mask
    return probs / (probs.sum() + 1e-10)

def apply_top_p(probs, p):
    idx = np.argsort(probs)[::-1]
    cumulative = np.cumsum(probs[idx])
    # Encontra onde a soma acumulada passa de P
    cutoff_idx = np.where(cumulative > p)[0]
    if len(cutoff_idx) > 0:
        actual_cutoff = cutoff_idx[0] + 1
    else:
        actual_cutoff = len(probs)

    new_probs = np.zeros_like(probs)
    for i in range(actual_cutoff):
        new_probs[idx[i]] = probs[idx[i]]
    return new_probs / (new_probs.sum() + 1e-10)

# ---------------------------------------------------------
# 2. INTEGRAÇÃO COM O SEU DecoderOnlyTransformer
# ---------------------------------------------------------
def get_logits_from_model(text):
    model.eval()

    # Detectar o device automaticamente
    device = next(model.parameters()).device

    # Tokenização (ajustado para o seu objeto tokenizer)
    enc = tokenizer.encode(text)
    ids = enc.ids if hasattr(enc, 'ids') else enc

    # Truncar se for maior que a capacidade do modelo (max_len)
    if len(ids) > model.max_len:
        ids = ids[-model.max_len:]

    input_ids = torch.tensor([ids]).to(device)

    with torch.no_grad():
        # O seu modelo retorna os logits diretamente como um tensor [B, T, Vocab]
        logits_output = model(input_ids)
        # Pegamos apenas os logits do último token gerado: [Vocab]
        last_logits = logits_output[0, -1, :]

    # Pegamos os top 10 maiores para a visualização inicial
    top_v, top_i = torch.topk(last_logits, 10)

    # Decodificação dos tokens para labels do gráfico
    tokens = []
    for idx in top_i.tolist():
        try:
            t = tokenizer.decode([idx])
            tokens.append(t if t.strip() != "" else f"ID:{idx}")
        except:
            tokens.append(f"ID:{idx}")

    return tokens, top_v.cpu().numpy()

# ---------------------------------------------------------
# 3. INTERFACE E GRÁFICO
# ---------------------------------------------------------
prompt_input = widgets.Text(
    value="O sol está",
    placeholder="Digite o início de um texto...",
    description="Prompt:",
    layout=widgets.Layout(width="95%")
)

temp_slider = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="Temp")
k_slider = widgets.IntSlider(value=10, min=1, max=10, step=1, description="Top-K")
p_slider = widgets.FloatSlider(value=1.0, min=0.1, max=1.0, step=0.05, description="Top-P")

output_plot = widgets.Output()

def update_viz(*args):
    with output_plot:
        output_plot.clear_output()
        if not prompt_input.value.strip(): return

        try:
            tokens, base_logits = get_logits_from_model(prompt_input.value)

            # Aplica transformações de amostragem
            logits = apply_temperature(base_logits, temp_slider.value)
            probs = softmax(logits)
            probs = apply_top_k(probs, k_slider.value)
            probs = apply_top_p(probs, p_slider.value)

            # Plotagem
            fig, ax = plt.subplots(figsize=(10, 4))
            bars = ax.bar(tokens, probs * 100, color='#6200ee')
            ax.set_ylim(0, 105)
            ax.set_ylabel("Probabilidade (%)")
            ax.set_title(f"Previsão do próximo token")

            for bar, p in zip(bars, probs):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f"{p*100:.1f}%", ha='center', va='bottom', fontsize=9)

            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Erro na inferência: {e}")

# Observadores
prompt_input.observe(update_viz, names="value")
for w in [temp_slider, k_slider, p_slider]:
    w.observe(update_viz, names="value")

# Exibição Final
display(Markdown("### 🧠 Visualizador de Inferência: Decoder Transformer"))
display(widgets.VBox([
    prompt_input,
    widgets.HBox([temp_slider, k_slider, p_slider]),
    output_plot
]))

# Execução inicial
update_viz()

### 🧠 Visualizador de Inferência: Decoder Transformer